<a href="https://colab.research.google.com/github/leonorsilva/Reinforcement-learning/blob/main/Boat_ppo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import math
from collections import deque
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
def isInBounds(x, y, width, height):
    if x>=0 and x<height and y>=0 and y<width:
        return True

    return False


In [ ]:
class Environment():
    def env_init(self, env_info={}):
        reward = None
        self.grid_orig=env_info.get('grid')
        self.grid=self.grid_orig.copy()
        termination = None
        self.start=env_info.get('start')
        self.goal=env_info.get('end')
        self.grid_h=env_info.get('grid_h')
        self.grid_w=env_info.get('grid_w')
        self.size=env_info.get('size')

        #self.direction=math.atan2(self.goal[1]-self.agent_loc[1],self.goal[0]-self.agent_loc[0])

        #self.rot=env_info.get('rot')   #radious
        #self.vel=0

        self.maxdist=(abs(self.grid_h)+abs(self.grid_w))
        self.reward_state_term = (reward, None, None, termination)



    def env_start(self):
        reward=0
        self.agent_loc = (np.random.randint(self.grid_w),np.random.randint(self.grid_h))
        self.goal=(np.random.randint(self.grid_w),np.random.randint(self.grid_h))
        self.sine=self.agent_loc[0]-self.goal[0]
        #math.asin((self.agent_loc[0]-self.goal[0])/self.hipotenusa)
        self.cosi=self.agent_loc[1]-self.goal[1]
        #math.acos((self.agent_loc[1]-self.goal[1])/self.hipotenusa)

        self.grid[self.goal[0]][self.goal[1]]=255
        #divide by pi to normalize value
        termination = False
        x, y = self.agent_loc
        if y==self.goal[1] and x==self.goal[0]:
            reward=1
            termination=True

        self.reward_state_term = (reward,self.sine,self.cosi, termination)

        return (self.sine,self.cosi,termination)

    def env_step(self,action):   #actions: [up,left,down,right]
        x, y = self.agent_loc

        if action==0:
            x=x-1
        elif action == 1:
            y = y - 1
        elif action == 2:
            x = x + 1
        elif action == 3:
            y = y + 1


        terminal = False

        reward=0  #-((abs(x-self.goal[0])+abs(y-self.goal[1]))/(self.grid_h+self.grid_w))*100

        if not isInBounds(x, y, self.grid_w, self.grid_h):
            x, y = self.agent_loc
            reward=-1
            terminal=True

        if self.grid[x][y]==0:
            x, y = self.agent_loc
            reward=0

        if y==self.goal[1] and x==self.goal[0]:
            reward=1
            terminal=True

        self.agent_loc = [x, y]

        self.sine=self.agent_loc[0]-self.goal[0]
        self.cosi=self.agent_loc[1]-self.goal[1]


        self.reward_state_term = (reward,self.sine,self.cosi, terminal)

        return self.reward_state_term

    def env_restart(self):
        """Cleanup done after the environment ends"""
        self.agent_loc = self.start
        self.grid=self.grid_orig.copy()

In [ ]:
#import numpy as np

#import torch
#import torch.nn as nn
#import torch.nn.functional as F

#device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


#class Actor_Critic(nn.Module):

#    def __init__(self, state_size, action_size, seed):

#        super(Actor_Critic, self).__init__()
#        self.seed = torch.manual_seed(seed)

        #action
        #self.fc1a = nn.Linear(101 , 64)
        #self.fc2a = nn.Linear(64, 32)
#        self.fc3a = nn.Linear(2, 32)
#        self.fc4a = nn.Linear(32, action_size)
#        self.sig=nn.Softmax(dim=1)

        #value
        #self.fc1v = nn.Linear(101 , 64)
        #self.fc2v = nn.Linear(64, 32)
#        self.fc3v = nn.Linear(2, 32)
#        self.fc4v = nn.Linear(32, 1)


#    def forward(self, sine,cosi):
#        """Build an actor (policy) network that maps states -> actions."""


#        x = torch.cat([sine,cosi],dim=1)
        #x=self.norm(x)

        #action
#        aux=F.sigmoid(self.fc3a(x))
#        aux=F.sigmoid(self.fc4a(aux))
#        probs=self.sig(aux)
        #logits=torch.logit(aux)
        #print(logits)
#        dist = torch.distributions.Categorical(probs=probs)

#        action = dist.sample()

#        log_prob = dist.log_prob(action).unsqueeze(-1)
#        entropy = dist.entropy().unsqueeze(-1)

        #value
#        x=F.tanh(self.fc3v(x))
#        v=F.tanh(self.fc4v(x))

#        return action,v,log_prob,entropy,probs

#    def action(self,sine,cosi,action):
 #       """Build an actor (policy) network that maps states -> actions."""


#        x = torch.cat([sine,cosi],dim=1)
        #x=self.norm(x)

        #action
#        aux=F.sigmoid(self.fc3a(x))
#        aux=F.sigmoid(self.fc4a(aux))
#        probs=self.sig(aux)
        #logits=torch.logit(aux)
        #print(logits)
#        dist = torch.distributions.Categorical(probs=probs)


#        log_prob = dist.log_prob(action).unsqueeze(-1)
#        entropy = dist.entropy().unsqueeze(-1)

        #value
#        x=F.tanh(self.fc3v(x))
#        v=F.tanh(self.fc4v(x))

#        return action,v,log_prob,entropy,probs


In [ ]:
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


class Actor(nn.Module):

    def __init__(self, state_size, action_size, seed):

        super(Actor, self).__init__()
        self.seed = torch.manual_seed(seed)

        #action
        #self.fc1a = nn.Linear(101 , 64)
        #self.fc2a = nn.Linear(64, 32)
        self.norm=nn.BatchNorm1d(2)
        self.fc3a = nn.Linear(2, 32)
        self.fc4a = nn.Linear(32, action_size)
        self.sig=nn.Softmax(dim=1)


    def forward(self, sine,cosi):
        """Build an actor (policy) network that maps states -> actions."""


        x = torch.cat([sine,cosi],dim=1)
        #x=self.norm(x)

        #action
        x=self.norm(x)
        aux=F.sigmoid(self.fc3a(x))
        aux=F.sigmoid(self.fc4a(aux))
        probs=self.sig(aux)
        #logits=torch.logit(aux)
        #print(logits)
        dist = torch.distributions.Categorical(probs=probs)

        action = dist.sample()

        #log_prob = dist.log_prob(action).sum(-1).unsqueeze(-1)
        #print(log_prob)
        entropy = dist.entropy().unsqueeze(-1)
        log_prob=torch.index_select(probs,1,action).squeeze(-1)
        #print(log_prob)
        return action,log_prob,entropy,probs

    def action(self,sine,cosi,action):
        """Build an actor (policy) network that maps states -> actions."""


        x = torch.cat([sine,cosi],dim=1)
        #x=self.norm(x)

        #action
        x=self.norm(x)
        aux=F.sigmoid(self.fc3a(x))
        aux=F.sigmoid(self.fc4a(aux))
        probs=self.sig(aux)
        #logits=torch.logit(aux)
        #print(logits)
        dist = torch.distributions.Categorical(probs=probs)


        #log_prob = dist.log_prob(action).sum(-1).unsqueeze(-1)

        log_prob=torch.index_select(probs,1,action).squeeze(-1)

        entropy = dist.entropy().unsqueeze(-1)

        return action,log_prob,entropy,probs


In [ ]:
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


class Critic(nn.Module):

    def __init__(self, state_size, action_size, seed):

        super(Critic, self).__init__()
        self.seed = torch.manual_seed(seed)

        #value
        #self.fc1v = nn.Linear(101 , 64)
        #self.fc2v = nn.Linear(64, 32)
        self.norm=nn.BatchNorm1d(2)
        self.fc3v = nn.Linear(2, 32)
        self.fc4v = nn.Linear(32, 1)


    def forward(self, sine,cosi):
        """Build an actor (policy) network that maps states -> actions."""

        x = torch.cat([sine,cosi],dim=1)
        #x=self.norm(x)

        #value
        x=self.norm(x)
        x=F.tanh(self.fc3v(x))
        v=F.tanh(self.fc4v(x))

        return v



In [ ]:

import time

import numpy as np
import time
import torch
import torch.nn as nn
from torch.optim import Adam, Adadelta, RMSprop
from collections import namedtuple, deque


ENT_DECAY = 0.995

class PPO:
    """
        This is the PPO class we will use as our model in main.py
    """
    def __init__(self,env, state_size,action_size, **hyperparameters):
        """
            Initializes the PPO model, including hyperparameters.
            Parameters:
                policy_class - the policy class to use for our actor/critic networks.
                env - the environment to train on.
                hyperparameters - all extra arguments passed into PPO that should be hyperparameters.
            Returns:
                None
        """
        # Make sure the environment is compatible with our code
        # Initialize hyperparameters for training with PPO
        self._init_hyperparameters(hyperparameters)

        # Extract environment information
        self.env=env
        self.obs_dim = state_size
        self.act_dim = action_size


         # Initialize actor and critic networks
        self.Actor = Actor(self.obs_dim, self.act_dim,30)
        self.Critic = Critic(self.obs_dim, self.act_dim,30)
        #self.critic = Critic(self.obs_dim, self.act_dim,30)


        # Initialize optimizers for actor and critic
        self.lr_inicial=self.lr
        self.b_inital=0.9
        #self.optim = RMSprop(self.Actor_Critic.parameters(), lr=self.lr,eps= 1e-5,alpha=0.9,momentum=self.b_inital)
        self.optima = Adam(self.Actor.parameters(), lr=self.lr,eps= 1e-5) #,betas=(0.8,0.999))
        self.optimc = Adam(self.Critic.parameters(), lr=self.lr,eps= 1e-5)

        self.entropy_w=0.0
        self.critic_w=1
        self.target_kl=None

        self.avg_ret=[]
        self.random=1


        # Initialize the covariance matrix used to query the actor for actions
        self.batch=25

        # This logger will help us with printing out summaries of each iteration
        self.logger = {
            'delta_t': time.time(),
            't_so_far': 0,          # timesteps so far
            'i_so_far': 0,          # iterations so far
            'batch_lens': [],       # episodic lengths in batch
            'batch_rews': [],       # episodic returns in batch
            'actor_losses': [],     # losses of actor network in current iteration
            'critic_losses': [],
            'entropy_losses': [],
            'approx_kl_divs': [],
        }

    def learn(self, total_timesteps):
        """
            Train the actor and critic networks. Here is where the main PPO algorithm resides.
            Parameters:
                total_timesteps - the total number of timesteps to train for
            Return:
                None
        """
        print(f"Learning... Running {self.max_timesteps_per_episode} timesteps per episode, ", end='')
        print(f"{self.timesteps_per_batch} timesteps per batch for a total of {total_timesteps} timesteps")
        t_so_far = 0 # Timesteps simulated so far
        i_so_far = 0 # Iterations ran so far
        while i_so_far < total_timesteps:                                                                       # ALG STEP 2
            # Autobots, roll out (just kidding, we're collecting our batch simulations here)
            batch_sine, batch_cosi, batch_acts, batch_log_probs, batch_rtgs, batch_lens, dones, batch_advs = self.rollout()                     # ALG STEP 3


            # Calculate how many timesteps we collected this batch
            t_so_far += np.sum(batch_lens)

            # Increment the number of iterations
            i_so_far += 1

            # Logging timesteps so far and iterations so far
            self.logger['t_so_far'] = t_so_far
            self.logger['i_so_far'] = i_so_far

            # Calculate advantage at k-th iteration
            V, _ ,_= self.evaluate(batch_sine, batch_cosi, batch_acts)

            #_,Vnext,_,_ = self.actor_critic(batch_obs_next)

            #batch_rtgs=batch_rtgs+self.gamma*Vnext.detach()*dones

            print('return', np.sort(batch_rtgs.detach().numpy().reshape(1,-1)[0])[-10:])
            print('return', np.sort(batch_rtgs.detach().numpy().reshape(1,-1)[0])[:10])
            #VV=Vnext.cpu().detach().numpy().reshape(1,-1)[0]
            A_k = batch_advs
            A_k = (A_k - A_k.mean()) / (A_k.std() + 1e-10)
            #print('v', V.cpu().detach().numpy().reshape(1,-1)[0][-10:])
            #print('adv', A_k)

            # This is the loop where we update our network for some n epochs
            for epoch in range(self.n_updates_per_iteration):


                continue_training=True
                approx_kl_divs=[]
                for  _ in range(int(len(batch_acts)/self.batch)):
                    indices = np.random.permutation(len(batch_acts))
                    idx=indices[ :self.batch]
                    batch_acts2=batch_acts[idx]
                    batch_sine2=batch_sine[idx]
                    batch_cosi2=batch_cosi[idx]
                    batch_log_probs2=batch_log_probs[idx]
                    batch_rtgs2=batch_rtgs[idx]
                    A_k2=A_k[idx]


                    # Calculate V_phi and pi_theta(a_t | s_t)
                    V2, curr_log_probs,entropy = self.evaluate(batch_sine2,batch_cosi2, batch_acts2)

                    ratios = curr_log_probs / batch_log_probs2 #torch.exp(curr_log_probs - batch_log_probs2)

                    # Calculate surrogate losses.

                    surr1 = ratios * A_k2
                    surr2 = torch.clamp(ratios, 1 - self.clip, 1 + self.clip) * A_k2

                    #sum(-1)???
                    actor_loss = -torch.min(surr1, surr2).mean()
                    entropy_loss=0 #self.entropy_w*(-torch.mean(entropy))

                    critic_loss = self.critic_w*nn.L1Loss()(V2, batch_rtgs2)

                    total_loss=actor_loss+entropy_loss+critic_loss

                    with torch.no_grad():  #????
                        log_ratio = curr_log_probs - batch_log_probs2
                        approx_kl_div = torch.mean((torch.exp(log_ratio) - 1) - log_ratio).cpu().numpy()
                        approx_kl_divs.append(approx_kl_div)

                    if self.target_kl is not None and approx_kl_div > 1.5 * self.target_kl:
                        continue_training = False

                        print(f"Early stopping at step {epoch} due to reaching max kl: {approx_kl_div:.2f}")
                        #break

                    # Calculate gradients and perform backward propagation for critic network

                    self.optima.zero_grad()
                    actor_loss.backward()
                    #torch.nn.utils.clip_grad_norm(self.Actor_Critic.parameters(), 5)
                    self.optima.step()

                    self.optimc.zero_grad()
                    critic_loss.backward()
                    #torch.nn.utils.clip_grad_norm(self.Actor_Critic.parameters(), 5)
                    self.optimc.step()

                    # Log actor loss
                    self.logger['actor_losses'].append(actor_loss.detach().numpy())
                    #self.logger['entropy_losses'].append(entropy_loss.detach())
                    self.logger['critic_losses'].append(critic_loss.detach().numpy())

                #if not continue_training:
                #    break

            V, _ ,_= self.evaluate(batch_sine, batch_cosi,batch_acts)
            #print(V.cpu().detach().numpy().reshape(1,-1)[0][-10:])

            #lr =max(self.lr_inicial * (1-i_so_far/total_timesteps),0.0003)
            #for param_group in self.optim.param_groups:
            #    param_group['lr'] = lr


            # Print a summary of our training so far
            self.logger['approx_kl_divs'].append(np.mean(approx_kl_divs))

            self._log_summary()





    def rollout(self):
        """
            Too many transformers references, I'm sorry. This is where we collect the batch of data
            from simulation. Since this is an on-policy algorithm, we'll need to collect a fresh batch
            of data each time we iterate the actor/critic networks.
            Parameters:
                None
            Return:
                batch_obs - the observations collected this batch. Shape: (number of timesteps, dimension of observation)
                batch_acts - the actions collected this batch. Shape: (number of timesteps, dimension of action)
                batch_log_probs - the log probabilities of each action taken this batch. Shape: (number of timesteps)
                batch_rtgs - the Rewards-To-Go of each timestep in this batch. Shape: (number of timesteps)
                batch_lens - the lengths of each episode this batch. Shape: (number of episodes)
        """
        # Batch data. For more details, check function header.
        self.Actor.eval()
        self.Critic.eval()

        batch_acts = []
        batch_log_probs = []
        batch_rews = []
        batch_rtgs = np.array([])
        batch_lens = []
        batch_value=[]
        batch_sine=[]
        batch_cosi=[]
        dones=[]
        batch_advs=np.array([])


        # Episodic data. Keeps track of rewards per episode, will get cleared
        # upon each new episode
        ep_rews = []

        t = 0 # Keeps track of how many timesteps we've run so far this batch


        # Keep simulating until we've run more than or equal to specified timesteps per batch
        while t < self.timesteps_per_batch:
            self.random=np.random.rand()
            print(self.random)
            ep_rews2 = [] # rewards collected per episode

            # Reset the environment. sNote that obs is short for observation.
            sine,cosi,done = env.env_start()

            X=[]
            Y=[]
            ep_t=0
            # Run an episode for a maximum of max_timesteps_per_episode timesteps
            #for ep_t in range(self.max_timesteps_per_episode):
            while not done:
                ep_t+=1
                # If render is specified, render the environment
                if self.render and (self.logger['i_so_far'] % self.render_every_i == 0) and len(batch_lens) == 0:
                    self.env.render()

                #t += 1 # Increment timesteps ran this batch so far
                t+=1

                # Track observations in this batch

                #obs = torch.from_numpy(np.array([obs])).float().to(device)
                sine=torch.from_numpy(np.array([[sine]])).float()
                cosi=torch.from_numpy(np.array([[cosi]])).float()


                batch_sine.append(sine)
                batch_cosi.append(cosi)


                # Calculate action and make a step in the env.
                # Note that rew is short for reward.
                action, log_prob,value,_= self.get_action(sine,cosi)
                rew,sine,cosi,done=env.env_step(action)
                #obs,rew,done,_= self.env.step(action.flatten())

                # Track recent reward, action, and action log probability
                ep_rews.append(rew)
                ep_rews2.append(rew)
                batch_acts.append(action)
                batch_value.append(value.detach().numpy()[0])
                batch_log_probs.append(log_prob)
                X.append(self.env.agent_loc[0])
                Y.append(self.env.agent_loc[1])

                # If the environment tells us the episode is terminated, break
                if done:
                    dones.append(1)
                    break
                dones.append(0)


            # Track episodic lengths and rewards
            batch_lens.append(ep_t + 1)
            #batch_rews.append(ep_rews2)
            self.logger['batch_rews'].append(ep_rews2)
            plt.imshow(self.env.grid, cmap='gray')
            plt.scatter(Y,X,color='r')
            plt.scatter(self.env.goal[1],self.env.goal[0],color='b')
            plt.scatter(self.env.agent_loc[1],self.env.agent_loc[0],color='g')
            plt.show()
            print(batch_value[-len(ep_rews2):])


            batch_advs2,batch_rtgs2 = self.compute_advs(ep_rews2,batch_value[-len(ep_rews2):])
            print('return',batch_rtgs2)
            print('gae',batch_advs2)
            batch_advs=np.append(batch_advs,batch_advs2)
            batch_rtgs=np.append(batch_rtgs,batch_rtgs2)

        #batch_value.append(0)


        # Reshape data as tensors in the shape specified in function description, before returning
        batch_acts = torch.tensor(batch_acts, dtype=torch.float)


        batch_log_probs=torch.tensor(batch_log_probs, dtype=torch.float)

        batch_sine=torch.tensor(np.vstack(batch_sine), dtype=torch.float)
        batch_cosi=torch.tensor(np.vstack(batch_cosi), dtype=torch.float)

        #batch_rews2=np.array(batch_rews)/(np.max(ep_rews)+1e-5)
        #batch_rews2=torch.tensor(batch_rews, dtype=torch.float)

        #torch.from_numpy(np.vstack(ep_rews)/(np.max(ep_rews)+1e-5)).float().to(device)

        dones=torch.tensor(dones, dtype=torch.float)
        #print(batch_advs)

        batch_advs=torch.tensor(np.vstack(batch_advs), dtype=torch.float)   #torch.tensor(np.vstack(batch_advs), dtype=torch.float)
        batch_rtgs=torch.tensor(np.vstack(batch_rtgs), dtype=torch.float)

        #self.logger['batch_lens'].append([np.sum(ep_rews) for ep_rews in self.logger['batch_rews']])

        self.Actor.train()
        self.Critic.train()

        return batch_sine, batch_cosi, batch_acts, batch_log_probs, batch_rtgs, batch_lens,dones,batch_advs



    def compute_advs(self, batch_rews,batch_value):
        with torch.no_grad():
            advantages = []
            returns = []

            size=0

            aux=[]
            last_gae_lam = 0

            next_non_terminal=next_values=0
            returne=0

            for rew in reversed(range(len(batch_rews))):


                returne=batch_rews[rew] + self.gamma * next_values * next_non_terminal

                delta = batch_rews[rew] + self.gamma * next_values * next_non_terminal - batch_value[rew]
                #last_gae_lam = delta + self.gamma * self.gae_lambda * next_non_terminal * last_gae_lam
                advantages.insert(0, delta )   #VERIFICAR GAE
                returns.insert(0,returne)
                next_non_terminal = 1

                next_values = batch_value[rew]

        return advantages,returns


    def get_action(self,sine,cosi,train=True ):
        """
            Queries an action from the actor network, should be called from rollout.
            Parameters:
                obs - the observation at the current timestep
            Return:
                action - the action to take, as a numpy array
                log_prob - the log probability of the selected action in the distribution
        """
        # Query the actor network for a mean action
        if np.random.rand()<self.random and train:
          #print(self.env.goal,self.env.agent_loc)
          #if direction>0 and direction<=np.pi/2:   #1quadrante

           # action=np.random.choice([3,0],1)
            #print(direction,'1',action)
          #elif direction>np.pi/2 and direction<=np.pi: #2quadrante

           # action=np.random.choice([1,0],1)
            #print(direction,'2',action)
          #elif direction<0 and direction>=-np.pi/2: #4quadrante

          #  action=np.random.choice([3,2],1)
            #print(direction,'4',action)
          #else:  #3quadrante

           # action=np.random.choice([2,1],1)
            #print(direction,'3',action)
          action=np.random.choice([0,3,2,1],1)
          action=torch.from_numpy(action)
          action,log_prob,entropy,dist = self.Actor.action(sine,cosi,action)
          value = self.Critic(sine,cosi)

        else:
          action,log_prob,entropy,dist = self.Actor(sine,cosi)
          value = self.Critic(sine,cosi)

          #log_prob = dist.log_prob(action)
        action=action.detach().numpy()
        log_prob = log_prob.detach().numpy()


        return action, log_prob,value[0],dist

    def play(self):
        self.Actor.eval()
        self.Critic.eval()
        sine,cosi,done = env.env_start()
        print(env.agent_loc)
        while not done:

            sine=np.array([[sine]])
            cosi=np.array([[cosi]])
            sine=torch.from_numpy(sine).float()
            cosi=torch.from_numpy(cosi).float()

            action, log_prob,value,dist= self.get_action(sine,cosi,train=False)
            rew,sine,cosi,done=env.env_step(action)
            print(action,dist,value,rew,log_prob)
            plt.imshow(self.env.grid, cmap='gray')
            plt.scatter(self.env.goal[1],self.env.goal[0],color='b')
            plt.scatter(self.env.agent_loc[1],self.env.agent_loc[0],color='g')
            plt.show()



    def evaluate(self,batch_sine, batch_cosi, batch_acts):
        """
            Estimate the values of each observation, and the log probs of
            each action in the most recent batch with the most recent
            iteration of the actor network. Should be called from learn.
            Parameters:
                batch_obs - the observations from the most recently collected batch as a tensor.
                            Shape: (number of timesteps in batch, dimension of observation)
                batch_acts - the actions from the most recently collected batch as a tensor.
                            Shape: (number of timesteps in batch, dimension of action)
            Return:
                V - the predicted values of batch_obs
                log_probs - the log probabilities of the actions taken in batch_acts given batch_obs
        """
        # Query critic network for a value V for each batch_obs. Shape of V should be same as batch_rtgs
        action,log_prob,entropy,_ = self.Actor(batch_sine, batch_cosi)
        value = self.Critic(batch_sine, batch_cosi)

        #log_prob = dist.log_prob(batch_acts.squeeze())


        return value.reshape(1,-1)[0], log_prob,entropy


    def _init_hyperparameters(self, hyperparameters):
        """
            Initialize default and custom values for hyperparameters
            Parameters:
                hyperparameters - the extra arguments included when creating the PPO model, should only include
                                    hyperparameters defined below with custom values.
            Return:
                None
        """
        # Initialize default values for hyperparameters
        # Algorithm hyperparameters
        self.timesteps_per_batch = 1000                 # Number of timesteps to run per batch
        self.max_timesteps_per_episode = 600           # Max number of timesteps per episode
        self.n_updates_per_iteration = 10                # Number of times to update actor/critic per iteration
        self.lr = 0.001                                 # Learning rate of actor optimizer
        self.gamma = 0.99                               # Discount factor to be applied when calculating Rewards-To-Go
        self.clip = 0.2                                 # Recommended 0.2, helps define the threshold to clip the ratio during SGA

        # Miscellaneous parameters
        self.render = False                              # If we should render during rollout
        self.render_every_i = 10                        # Only render every n iterations
        self.save_freq = 10                             # How often we save in number of iterations
        self.seed = None                                # Sets the seed of our program, used for reproducibility of results

        # Change any default values to custom values for specified hyperparameters
        for param, val in hyperparameters.items():
            exec('self.' + param + ' = ' + str(val))

        # Sets the seed if specified
        if self.seed != None:
            # Check if our seed is valid first
            assert(type(self.seed) == int)

            # Set the seed
            torch.manual_seed(self.seed)
            print(f"Successfully set seed to {self.seed}")

    def _log_summary(self):
        """
            Print to stdout what we've logged so far in the most recent batch.
            Parameters:
                None
            Return:
                None
        """
        # Calculate logging values. I use a few python shortcuts to calculate each value
        # without explaining since it's not too important to PPO; feel free to look it over,
        # and if you have any questions you can email me (look at bottom of README)
        delta_t = self.logger['delta_t']
        self.logger['delta_t'] = time.time()
        delta_t = (self.logger['delta_t'] - delta_t) / 1e9
        delta_t = str(round(delta_t, 2))

        t_so_far = self.logger['t_so_far']
        i_so_far = self.logger['i_so_far']
        avg_ep_lens = np.mean(self.logger['batch_lens'][-100:])
        avg_ep_rews = np.mean([np.sum(ep_rews) for ep_rews in self.logger['batch_rews']][-100:])
        max_ret=np.max([np.sum(ep_rews) for ep_rews in self.logger['batch_rews']])
        avg_actor_loss = np.mean([np.mean(self.logger['actor_losses'])])
        avg_critic_loss = np.mean([np.mean(self.logger['critic_losses'])])
        avg_entropy_loss = np.mean([np.mean(self.logger['entropy_losses'])])



        self.avg_ret.append(np.round(avg_ep_rews,2))
        # Round decimal places for more aesthetic logging messages
        avg_ep_lens = str(round(avg_ep_lens, 2))
        avg_ep_rews = str(round(avg_ep_rews, 2))
        avg_actor_loss = str(round(avg_actor_loss, 5))

        # Print logging statements
        print(flush=True)
        print(f"-------------------- Iteration #{i_so_far} --------------------", flush=True)
        print(f'random ',self.random)
        print(f"Average : {self.lr_inicial}", flush=True)
        print(f"Average Episodic Return: {avg_ep_rews}", flush=True)
        print(f"Max Episodic Return: {max_ret}", flush=True)
        print(f"Average Loss: {avg_actor_loss}", flush=True)
        print(f"Average Loss Critic: {avg_critic_loss}", flush=True)
        #print(f"STD: {std}", flush=True)
        print(f"TEntropy: {avg_entropy_loss}", flush=True)
        print(f"approx_kl_divs: {self.logger['approx_kl_divs']} secs", flush=True)
        print(f"------------------------------------------------------", flush=True)
        print(flush=True)

        # Reset batch-specific logging data
        #self.logger['batch_rews'] = []

        self.logger['approx_kl_divs']=[]



In [ ]:
#train easy mode
img_matrix_filtered2=np.ones((6,6))*125
startlat=1
startlon=1
endlon=5
endlat=5

In [ ]:

env=Environment()
env_info={'grid':img_matrix_filtered2,'grid_h':img_matrix_filtered2.shape[0],'grid_w':img_matrix_filtered2.shape[1],
          'size':5,'start':[startlat,startlon],'end':[endlat,endlon]}
env.env_init(env_info)

In [ ]:
hyperparameters = {
                'gamma': 0.98 ,
                'gae_lambda':0.8,
                'lr': 0.01,
                'timesteps_per_batch':50,
                'max_timesteps_per_episode':500,
                'total_timesteps':100,
                'n_updates_per_iteration':5,
                'target_update_interval':1,
                'clip': 0.2,
                'render': False,
                'render_every_i': 10
              }
agent=PPO(env,6*6, 4, random_seed=10,**hyperparameters)

In [ ]:
agent.learn(500)

In [ ]:
plt.plot(agent.logger['actor_losses'],'.' )

In [ ]:
agent.play()

In [ ]:
torch.save(agent.Actor_Critic.state_dict(),'weihts.pth')

In [ ]:
agent.Actor_Critic.state_dict()